In [1]:
from elasticsearch import Elasticsearch
import json
import openai
from openai import OpenAI
import os
from tqdm.auto import tqdm

/workspaces/llm-zoomcamp-dm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
es_client = Elasticsearch("http://127.0.0.1:9200", api_key="8.17.6")

In [ ]:
# with open('documents.json', 'rt') as f_in:
#     documents_raw = json.load(f_in)

In [ ]:
# documents = []

# for course_dict in documents_raw:
#     for doc in course_dict['documents']:
#         doc['course'] = course_dict['course']
#         documents.append(doc)

In [6]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course_questions'

# we created the index with the settings above
es_client.indices.create(
    index=index_name,
    body=index_settings
)

# now we are indexing the documents using the created indices
for doc in tqdm(documents):
    es_client.index(
        index=index_name,
        document=doc
    )

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course_questions/9TakxejtR3qLtV5XZPmFuQ] already exists')

In [ ]:
query = "how do i run kafka?"

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    search_results = es_client.search(index=index_name, body=search_query)

    # print(search_results)

    results=[]

    for i in search_results['hits']['hits']:
        results.append(i['_source'])

    return results

In [9]:
def build_prompt(results, query):
    prompt_template = """
    You are a helpful educational assistant.
    based on the given Context and the user_query provide a contextual and relevant answer to the user_query.
    If the user_query is not in the Context, say that you don't know the answer.

    user_query:{query}

    Context = {results}
    """
    context = ""
    for i in results:
        context += f"\n\n question:{i['question']} \n Answer:{i['text']}\n"

    return prompt_template.format(query=query, results=context)

In [10]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user','content':prompt}]
    )
    return response.choices[0].message.content.strip()

In [11]:
def rag(query):
    results = elastic_search(query)
    prompt = build_prompt(results,query)
    response = llm(prompt)
    return response

In [25]:
elastic_search("How do execute a command on a Kubernetes pod?")

{'took': 37, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 334, 'relation': 'eq'}, 'max_score': 23.980145, 'hits': [{'_index': 'course_questions', '_id': 'iQXodZcBueLjnQtIASUg', '_score': 23.980145, '_source': {'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:', 'section': 'Course Management Form for Homeworks', 'question': 'How to run a dbt-core project as an Airflow Task Group on Google Cloud 

[{'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:',
  'section': 'Course Management Form for Homeworks',
  'question': 'How to run a dbt-core project as an Airflow Task Group on Google Cloud Composer using a service account JSON key',
  'course': 'data-engineering-zoomcamp'},
 {'text': "There are 3 Zoom Camps in a year, as of 2024. However, they are for separate courses:\nData-Engineering (Jan - Apr)\nMLOps (May - Aug)\nMachine Learning (Sep - Jan)\nThere's only 

In [19]:
hw3

[{'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:',
  'section': 'Course Management Form for Homeworks',
  'question': 'How to run a dbt-core project as an Airflow Task Group on Google Cloud Composer using a service account JSON key',
  'course': 'data-engineering-zoomcamp'},
 {'text': "There are 3 Zoom Camps in a year, as of 2024. However, they are for separate courses:\nData-Engineering (Jan - Apr)\nMLOps (May - Aug)\nMachine Learning (Sep - Jan)\nThere's only 